In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Resizing, Flatten, Cropping2D, Add
from tensorflow.keras.models import Model

In [2]:
class StreamBlock(Model):

    def __init__(self):
        super(StreamBlock, self).__init__(name='stream')
        self.conv1 = Conv2D(96, 7, activation='relu')
        self.pool1 = MaxPool2D((2, 2))
        self.conv2 = Conv2D(160, 3, activation='relu')
        self.pool2 = MaxPool2D((2, 2))
        self.conv3 = Conv2D(288, 3, activation='relu')
        self.pool3 = MaxPool2D((2, 2))
        self.flatten = Flatten()
        self.dense = Dense(512, activation='relu')

    def call(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.dense(x)

In [3]:
class MrCNN(Model):

    def __init__(self):
        super(MrCNN, self).__init__(name='mr_cnn')
        self.resize_1 = Resizing(400, 400)
        self.resize_2 = Resizing(250, 250)
        self.resize_3 = Resizing(150, 150)
        self.stream_1 = StreamBlock()
        self.stream_2 = StreamBlock()
        self.stream_3 = StreamBlock()
        self.dense = Dense(512, activation='relu')
        self.add = Add()
        self.classifier = Dense(2, activation='sigmoid')

    def call(self, input_tensor):
        s1 = self.resize_1(input_tensor)
        # s1 = Cropping2D()(s1)
        s1 = self.stream_1(s1)

        s2 = self.resize_2(input_tensor)
        # s2 = Cropping2D()(s2)
        s2 = self.stream_1(s2)

        s3 = self.resize_3(input_tensor)
        # s3 = Cropping2D()(s3)
        s3 = self.stream_1(s3)

        output = self.add([s1, s2, s3])
        output = self.dense(output)
        output = self.classifier(output)


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(featurewise_center=False)

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('torronto_df.csv')
df.is_salient = df.is_salient.astype(str)
df.center_index = df.center_index.apply(eval).apply(np.array)
df.head()

,Unnamed: 0,image_file,fixation_file,image_size,center_index,is_salient
0,0,data/toronto/fixdens/images/1.jpg,data/toronto/fixdens/output/d1.jpg,"(681, 511)","[376, 367]",1
1,1,data/toronto/fixdens/images/1.jpg,data/toronto/fixdens/output/d1.jpg,"(681, 511)","[380, 351]",1
2,2,data/toronto/fixdens/images/1.jpg,data/toronto/fixdens/output/d1.jpg,"(681, 511)","[375, 349]",1
3,3,data/toronto/fixdens/images/1.jpg,data/toronto/fixdens/output/d1.jpg,"(681, 511)","[383, 351]",1
4,4,data/toronto/fixdens/images/1.jpg,data/toronto/fixdens/output/d1.jpg,"(681, 511)","[375, 351]",1


In [6]:
# def generate_custom_data(target_size=df.image_size[0]):
#     gen_1 = datagen.flow_from_dataframe(df,
#                                         x_col='image_file',
#                                         y_col='is_salient',
#                                         class_mode='binary',
#                                         shuffle=False,
#                                         target_size=target_size,
#                                         seed=121)
#     gen_2 = datagen.flow_from_dataframe(df,
#                                         x_col='image_file',
#                                         y_col='center_index',
#                                         class_mode='raw',
#                                         shuffle=False,
#                                         target_size=target_size,
#                                         seed=121)
#     while True:
#         X1i = gen_1.next()
#         X2i = gen_2.next()
        
#         yield [X1i[0], X2i[1]], X1i[1]


In [7]:
def generate_custom_data(target_size):
    # gen_1 = datagen.flow_from_dataframe(df,
    #                                     x_col='image_file',
    #                                     y_col='is_salient',
    #                                     class_mode='binary',
    #                                     shuffle=False,
    #                                     target_size=target_size,
    #                                     seed=121)
    gen_2 = datagen.flow_from_dataframe(df,
                                        x_col='image_file',
                                        y_col=['is_salient', 'center_index'],
                                        class_mode='raw',
                                        shuffle=False,
                                        target_size=target_size,
                                        batch_size=64,
                                        seed=121)
    for batch in gen_2:
        print(batch)
        input_batch, output_batch = [batch[0], batch[1][:,1]], batch[1][:,0]
        yield input_batch, output_batch

In [8]:
# train_gen = generate_custom_data()

In [9]:
gen = generate_custom_data((500,500))

In [10]:
mr_cnn = MrCNN()

In [11]:
mr_cnn.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
mr_cnn.fit(gen, epochs=10)

Found 3600 validated image filenames.
(array([[[[ 94.,  80.,  77.],
         [121., 107., 104.],
         [115., 101.,  98.],
         ...,
         [131., 135., 134.],
         [132., 134., 133.],
         [132., 134., 133.]],

        [[ 95.,  81.,  78.],
         [122., 108., 105.],
         [113., 102.,  98.],
         ...,
         [132., 136., 135.],
         [132., 134., 133.],
         [138., 140., 139.]],

        [[ 96.,  85.,  81.],
         [121., 110., 106.],
         [114., 103.,  99.],
         ...,
         [132., 136., 135.],
         [132., 134., 133.],
         [127., 129., 128.]],

        ...,

        [[ 79.,  51.,  30.],
         [ 78.,  50.,  29.],
         [ 79.,  53.,  30.],
         ...,
         [ 82.,  65.,  39.],
         [ 77.,  61.,  36.],
         [ 89.,  74.,  55.]],

        [[ 21.,   0.,   0.],
         [ 21.,   0.,   0.],
         [ 20.,   0.,   0.],
         ...,
         [ 25.,   9.,   0.],
         [ 28.,  13.,   0.],
         [ 11.,   0.,   0.]]

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [ ]:
gen_1 = datagen.flow_from_dataframe(df,
                                    x_col='image_file',
                                    y_col='center_index',
                                    class_mode='raw',
                                    shuffle=False,
                                    target_size=(500,500),
                                    seed=121)

: 

In [ ]:
a = gen_1.next()
a[0].shape, 

: 

In [ ]:
a[1].shape

: 

In [ ]:
a[1]

: 

In [ ]:
gen_1 = datagen.flow_from_dataframe(df,
                                    x_col='image_file',
                                    y_col='is_salient',
                                    class_mode='binary',
                                    shuffle=False,
                                    target_size=(500,500),
                                    seed=121)
gen_2 = datagen.flow_from_dataframe(df,
                                    x_col='image_file',
                                    y_col='center_index',
                                    class_mode='raw',
                                    shuffle=False,
                                    target_size=(500,500),
                                    seed=121)

: 

In [ ]:
gen_1.next()

: 